# 2020 유행어

In [1]:
import pandas as pd
import numpy as np
from kiwipiepy import Kiwi
import copy
import re
from kiwipiepy.utils import Stopwords
import nltk

from wordcloud import WordCloud # 워드클라우드 제작 라이브러리
import pandas as pd # 데이터 프레임 라이브러리
import numpy as np # 행렬 라이브러리
import matplotlib.pyplot as plt # 워드클라우드 시각화 라이브러리
%matplotlib inline
import konlpy

In [2]:
df = pd.read_csv('2020년 유행어.csv', encoding='utf-8')

# postdate, body, description, title 칼럼만 사용 
df = df[['postdate', 'body', 'description', 'title']]
df.head()

,postdate,body,description,title
0,20210106,"제가 매년 연말마다 찾아보는 것 중에 하나예요. 일본에서는 매년 그해에 유행했거나,...",youtube.com\/channel\/UC1B51m7HSWGpm_qDDgoIeqA...,"2020년 일본 유행어,신조어 대상 best 10."
1,20210105,"안녕! 아이비 친구들! 자, 오늘은 인싸력 레벨을 한 번 테스트해볼까? 2021년이...","2021년이 온 기념으로, 2020년 한 해를 빛낸 유행어는 어떤 것들이 있었는지 ...",[인싸력 테스트] 2020년을 빛낸 유행어 알아보기
2,20201218,"안녕하세요, 멀리 보여도 늘 가까운 이웃나라, 일본. 핫하고 속 깊은 일본 소식을 ...",com\/2020\/04\/2165\/) 매년 12월이 되면 한 해 동안 유행했던 ...,2020년 유행어 대상(2020年 流行語大賞)
3,20201224,인터넷을 하다보면 처음 보는 줄임말이나 유행어들이 자주 보이는데요! 중국에는 어떤 ...,청년문적이 발표한 ‘<b>2020년<\/b> 올해 인터넷에서 가장 반응이 뜨거웠던 ...,2020년 많이 쓰인 중국의 유행어 및 인터넷 용어
4,20200601,"으아 일하기시러 바로 시작 ㄹㅇㅋㅋ 혐오의 시대에 살고 있는 요즘, 한 가지 댓글이...","크크루삥뽕 트위치발 <b>유행어<\/b>, 내지는 밈. 트위치는 도네이션, 아프리카...",2020년 유행어 트렌드 5월편 B급 주의


## 1. 전처리

In [3]:
## 사용자 정의 단어 추가 
kiwi = Kiwi()

new = ['스라밸', '쌉파서블', '2000원', '머선', '머선129', '인성', '문제', '갑분싸', '국룰', 
       '삼귀다', '자만추', '레게노', '커엽', '존잘', '존귀', '졸귀', '존예', 'tmi', 'tmt', '띵작', 
       '팩폭', '레알', 'jmt', '갑툭튀', '먹방', '댕댕이', '존맛탱', 'ㄹㅇ', 'ㄹㅇㅋㅋ', '곁들인', '근데', '이제', 
       '꼬마아가씨', '꼬마', '아가씨']

for i in new:
    kiwi.add_user_word(i,'NNP')

In [4]:
# 전처리할 칼럼들
columns = ['body', 'description', 'title']

# 중복 제거
df.drop_duplicates(subset=columns, inplace=True)

for column in columns:
    # 정규표현식 - 한글, 숫자, 영어 및 공백 이외의 문자 제거
    df[column] = df[column].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]", "", regex=True)
    # 앞뒤 공백 제거
    df[column] = df[column].str.strip()
    # 여러 개의 공백을 하나의 공백으로 변경
    df[column] = df[column].str.replace(' +', " ", regex=True)

# 공백만 있는 칼럼을 NaN으로 변경 후 제거
df[columns] = df[columns].replace('', np.nan)
df.dropna(subset=columns, how='any', inplace=True)

In [5]:
from kiwipiepy import Kiwi

# Kiwi 초기화
kiwi = Kiwi()

# 불용어 리스트 정의 및 파일 로드
additional_stopwords = ['유행어', '신조어', '2020년', '생각', '올해', '시간', '사람', '코로나', '대하', '시작', 
                        '일본', '사용', '한국', '사랑', '때문', '중국', '사진', '미국', '사회', '시장' 
                        '의미', '영상', '단어', '표현', '블로그', '정도', '인기', '배우', '모습', '이야기', 
                        '활동', '세대', '부모', '출처', '세계', '마음', '친구', '가지', '유행', '행복', 
                        '작품', '출연',  '이후', '차트', '상황', '다양', '관련', '최고', '멤버', '제공',
                        '가능', '게임', '대상', '처음', '일상', '요즘', '보이', '여행', '진행', '프로그램', 
                        '시대', '앨범', '소개', '성공', '느낌', '정보', '콘텐츠','네이버', '오늘', '발매', 
                        '생활', '내용', '기업', '투자', '음악', '경제', '대표', '캐릭터', '시장', '광고', 
                        '트렌드', '이름', '부분', '엄마', '필요', '가족', '기억', '이유', '인터넷', '경우', 
                        '성장', '세상', '관심', '당시', '공부', '온라인', '감사', '마지막', '대화', '시즌', 
                        '등장', '기록', '사이', '준비', '콘서트', '인하', '영어', '공연', '데뷔', '발표', 
                        '공개', '개인', '나라', '중요', '자리', '회사', '여자', '서울', '도전', '교육', 
                        '기준', '채널', '운동', '순위', '대한민국', '변화', '소비', '제품', '경험', '가격', 
                        '마스크', '판매', '감독', '추천', '기술', '포스팅', '국내', '학교', '여성', '하루', 
                        '스타', '소통', '영국', '나이', '유명', '이미지', '청년', '건강', '카페', '방법', 
                        '이웃', '연기', '활용', '지금', '어머니', '정리', '운영', '작년', '소리', '남자', 
                        '인생', '서비스', '제작', '정부', '중국어', '시절', '노력', '기사', '하나님', '개그맨', 
                        '예수', '가수', '그룹', '화제',  '브랜드', '실패', '문장', '일본어', '이해', '거리', 
                        '언니', '진입', '최근', '결과', '물건','취미', '미래', '이용', '코미디', '아버지', 
                        '기대', '충격', '선택', '현실', '평가', '얼굴', '과거', '사건', '아이돌', '설명', 
                        '아래', '수업', '지역', '참여', '과정', '확인', '공간', '고민', '인간', '리뷰', 
                        '지원', '아빠', '아들', '예능', '시청', '커피', '개봉', '목표', '출시', '사실', 
                        '기분', '네이버', '의미', '대학', '환경', '부동산', '국민', '자체', '상태', '인물',
                        '방식', '해당', '플랫폼', '말씀', '장면','음식', '능력', '주식', '싱글', '자녀', 
                        '블로그', '기간', '선정', '릴레이', '산업', '본인', '수익', '주인공', '마케팅', '증가', 
                        '분야', '선물', '결혼', '국가', '순간','수상', '주목', '유튜버',  '구매', '존재', 
                        '검색', '영향', '수준', '행동', '대중', '디지털', '년대', '관계', '자연', '계획', 
                        '뉴스', '공감', '안녕하세요', '아침', '머리','용어', '작가', '선생', '도움', '소식', 
                        '질문', '라면', '가치', '관리', '결정', '코너', '분위기', '효과', '한류', '학생',
                        '언어', '동영상', '역사', '지목', '해외', '현상', '탄생', '현재' '예전', '주제', 
                        '감염', '무대', '중심', '데이터', '역할', '기회', '상승', '시기', '생산', '예정', 
                        '유지', '목소리', '개발', '제목', '기존', '공식', '단계', '사업', '소설', '언급', '얘기',
                        '스토리', '남편', '분석', '정책', '프로', '대신', '저녁', '부문', '시리즈', '주변', 
                        '인사', '코미디언', '소비자', '어워드', '선수','개월', '도시', '대사', '상품', '맥주', 
                        '방영', '리그', '모델', '기본', '전체', '스타일', '졸업', '인상', '인증', '바이러스',
                        '특징', '집중', '포함', '차이', '일반', '발생', '지식','발전', '발견', '직장', 
                        '한국어', '주류', '다이어리', '추억', '마무리', '글로벌', '오빠', '촬영', '반응', '예상', 
                        '경쟁', '전략', '걱정', '전문', '디자인','덕분', '구성', '지속', '센터', '이전', 
                        '조사', '이벤트', '공유', '행사', '출생', '정치', '시험', '시청자', '공경', '감정', 
                        '오랜만', '추가', '연출', '예술', '프로젝트', '규모', '형태', '정신', '논란', '사례',
                        '맛집', '본문', '부담', '상대', '이제', '확대','하늘', '확산', '일부', '부족', '인터뷰', 
                        '검사', '약속', '취업', '작성', '장르', '연결', '장소', '신청', '연예', '성격', 
                        '경기', '시스템', '프로필', '연예인', '여름', '흥행', '바람', '아파트', '비교', '달러', 
                        '고객', '가운데', '대로', '동생', '연말', '특별', '입장', '내년', '감동', '예측', 
                        '위치', '반영', '위기', '기획',  '가정', '새해', '동시', '생일', '번역', '패션', 
                        '판매량', '청소년', '위스키', '방문', '캠핑'
                        ]

# 불용어 텍스트 파일에서 불용어 읽기
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords_list = file.read().splitlines()
    return stopwords_list

# 파일에서 불용어 목록 로드
stopwords_file_path = '../stopwords.txt'
file_stopwords = load_stopwords(stopwords_file_path)

# 불용어 리스트 결합
all_stopwords = set(additional_stopwords + file_stopwords)

In [6]:
# 불용어 제거 함수 정의
def remove_stopwords(text, stopwords):
    tokens = kiwi.tokenize(text)
    filtered_tokens = [token.form for token in tokens if token.form not in stopwords]
    return ' '.join(filtered_tokens)

# 전처리 함수 정의
def preprocess_korean(text, analyzer=kiwi, stopwords=all_stopwords):
    my_text = copy.copy(text)
    my_text = my_text.replace('\n', ' ') # (1) 줄바꿈 문자 제거
    my_text = analyzer.space(my_text) # (2) 띄어쓰기 교정
    sents = analyzer.split_into_sents(my_text) # (3) 문장 토큰화
    p = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]')
    all_result = []
    for sent in sents:
        token_result = remove_stopwords(sent.text, stopwords) # (4) 형태소 분석 및 불용어 제거
        token_result = p.sub(' ', token_result) # (5) 특수 문자 제거 (=한글을 제외한 문자 제거)
        all_result.append(token_result) # (6) 형태소 분석한 결과를 다시 join
    
    all_result = ' '.join(all_result) # (7) 모든 문장을 하나의 string으로 join

    return all_result

# 품사(명사, 동사, 형용사, 부사) 추출 함수 정의
def wordclass_korean(my_str, kiwi=kiwi):
    result = []
    tokens = kiwi.tokenize(my_str, normalize_coda=True)
    for token in tokens:
        if token.tag in ['NNG', 'NNP', 'NNB']:  # 명사 태그만 추출
            result.append(token.form)
    result = ' '.join(result)
    return result

In [7]:
%%time
# 전처리 함수 적용
for column in columns:
    preprocessed_column = 'preprocessed_' + column 
    df[preprocessed_column] = df[column].apply(lambda x: preprocess_korean(x))

# 품사 추출 함수 적용
preprocessed_columns = ['preprocessed_body', 'preprocessed_title', 'preprocessed_description']
for column in preprocessed_columns:
    # wordclass_column = 'wordclass_' + column 
    df[column] = df[column].apply(wordclass_korean)

df.head()

: 

In [ ]:
# 전처리한 결과 csv로 저장 
df.to_csv('20유행어_전처리.csv', index=False, encoding='utf-8-sig')

## 2. 단어 빈도 계산

In [ ]:
# 단어 빈도 계산 및 상위 단어 추출 
def explode_and_count(df, column):
    exploded = df[column].str.split().explode()  # 문자열을 단어 단위로 나누어 행으로 펼침
    word_counts = exploded.value_counts().reset_index()  # 단어 빈도 계산
    word_counts.columns = ['word', 'count']  # 컬럼명 설정
    return word_counts


### [preprocessed_body]

In [ ]:
# 각 칼럼에 대해 단어 빈도 계산
word_counts_list = []
for column in ['preprocessed_body']:
    word_counts = explode_and_count(df, column)
    word_counts_list.append(word_counts)

In [ ]:
# 모든 칼럼의 단어 빈도 합산 및 정렬
all_words_body = pd.concat(word_counts_list).groupby('word').sum().reset_index().sort_values(by='count', ascending=False)

# 한 글자 단어 제외
all_words_body = all_words_body[all_words_body['word'].str.len() > 1]

# 상위 30개 단어 출력
top_30_words = all_words_body.head(200)
top_30_words

In [ ]:
l = []
for i in top_30_words['word']:
    l.append(i)
l

In [ ]:
## 한 번 해본 것

for i in all_words_body['word']:
    if i in new:
        count = all_words_body.loc[all_words_body['word'] == i, 'count'].values[0]
        print(f"The count for '{i}' is: {count}")

### [preprocessed_description]

In [ ]:
# 단어 빈도 계산 및 상위 단어 추출 
def explode_and_count(df, column):
    exploded = df[column].str.split().explode()  # 문자열을 단어 단위로 나누어 행으로 펼침
    word_counts = exploded.value_counts().reset_index()  # 단어 빈도 계산
    word_counts.columns = ['word', 'count']  # 컬럼명 설정
    return word_counts

# 각 칼럼에 대해 단어 빈도 계산
word_counts_list = []
for column in ['preprocessed_title']:
    word_counts = explode_and_count(df, column)
    word_counts_list.append(word_counts)

# 모든 칼럼의 단어 빈도 합산 및 정렬
all_word_counts = pd.concat(word_counts_list).groupby('word').sum().reset_index().sort_values(by='count', ascending=False)

# 한 글자 단어 제외
all_word_counts = all_word_counts[all_word_counts['word'].str.len() > 1]

# 상위 30개 단어 출력
top_30_words = all_word_counts.head(30)
top_30_words

### [preprocessed_title]

In [ ]:
# 단어 빈도 계산 및 상위 단어 추출 
def explode_and_count(df, column):
    exploded = df[column].str.split().explode()  # 문자열을 단어 단위로 나누어 행으로 펼침
    word_counts = exploded.value_counts().reset_index()  # 단어 빈도 계산
    word_counts.columns = ['word', 'count']  # 컬럼명 설정
    return word_counts

# 각 칼럼에 대해 단어 빈도 계산
word_counts_list = []
for column in ['preprocessed_description']:
    word_counts = explode_and_count(df, column)
    word_counts_list.append(word_counts)

# 모든 칼럼의 단어 빈도 합산 및 정렬
all_word_counts = pd.concat(word_counts_list).groupby('word').sum().reset_index().sort_values(by='count', ascending=False)

# 한 글자 단어 제외
all_word_counts = all_word_counts[all_word_counts['word'].str.len() > 1]

# 상위 30개 단어 출력
top_30_words = all_word_counts.head(30)
top_30_words

# 3. TF-IDF

In [ ]:
# ## 참고 사이트
# ## -> https://foreverhappiness.tistory.com/30
# ## -> https://foreverhappiness.tistory.com/35
# ## -> https://foreverhappiness.tistory.com/37
# ## 너무 오래 걸려서 안 돌림


# import pandas as pd
# from tqdm import tqdm
# from konlpy.tag import Okt
# from sklearn.feature_extraction.text import CountVectorizer

# # DTM (Document-Term Matrix) creation using Scikit-Learn's CountVectorizer
# # DTM(Document-Term Matrix) 생성 함수
# def NLP_DTM(df):
#     # 타이틀 리스트를 데이터프레임의 'preprocessed_title' 열에서 가져옴
#     title_lst = df['preprocessed_title']
    
#     # 불용어 리스트를 정의
#     stop_words_list = ['사람', '대하']
    
#     # Okt 형태소 분석기를 초기화
#     tagger = Okt()

#     # 결과를 저장할 DataFrame 초기화
#     result_df = pd.DataFrame()

#     # 타이틀 리스트를 순회하며 진행도를 표시
#     for title in tqdm(title_lst, desc='타이틀 리스트 진행도'):
#         # CountVectorizer 객체를 초기화
#         cv = CountVectorizer()

#         # 각 문서의 말뭉치를 저장할 리스트를 초기화
#         corpus = []

#         # 문서 진행도를 표시하며 타이틀 리스트의 각 요소에 대해 반복
#         for i in tqdm(range(len(df['preprocessed_body'])), desc='문서 진행도'):
#             # 각 타이틀에 대해 명사 리스트를 생성합니다.
#             n_lst = tagger.nouns(df['preprocessed_body'].iloc[i])
#             corpus.append(' '.join(n_lst))

#         # 말뭉치 데이터를 사용해 DTM(Document-Term Matrix)을 생성
#         DTM_array = cv.fit_transform(corpus).toarray()
        
#         # DTM의 각 열이 어떤 단어에 해당하는지 feature_names에 저장
#         feature_names = cv.get_feature_names_out()

#         # DTM 배열을 DataFrame 형식으로 변환
#         DTM_df = pd.DataFrame(DTM_array, columns=feature_names)
        
#         # 불용어 리스트에 있는 단어들을 DTM DataFrame에서 제거
#         DTM_df.drop(columns=stop_words_list, inplace=True, errors='ignore')
        
#         # 결과 DataFrame에 현재 DTM DataFrame을 추가
#         result_df = pd.concat([result_df, DTM_df], axis=0)

#     return result_df


# # %%time
# dtm_df = NLP_DTM(df)
# dtm_df

## 교수님 코드

In [ ]:
## DTM (Document Term Matrix)
## 이거 안 하고 바로 TF-IDF로 가도 됨

import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer


documents = df['preprocessed_body']
word_dict = all_word_counts['word']
vector = CountVectorizer(vocabulary=word_dict)
results = vector.fit_transform(documents).toarray()

results

In [ ]:
## TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

documents = df['preprocessed_body']
word_dict = all_word_counts['word']
vector = CountVectorizer(vocabulary=word_dict)
dtm = vector.fit_transform(documents).toarray()


vector = TfidfVectorizer(vocabulary=word_dict)
tfidf = vector.fit_transform(documents).toarray()

tfidf

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# 가정: df는 미리 전처리된 텍스트 데이터 프레임
documents = df['preprocessed_body']
word_dict = all_word_counts['word'].tolist()

# TF-IDF 계산
vectorizer = TfidfVectorizer(vocabulary=word_dict)
tfidf_matrix = vectorizer.fit_transform(documents).toarray()

# 각 단어의 TF-IDF 값을 집계
tfidf_scores = tfidf_matrix.sum(axis=0)
words = vectorizer.get_feature_names_out()

# 데이터 프레임 생성
tfidf_df = pd.DataFrame({'words': words, 'counts': tfidf_scores})

# TF-IDF 값이 큰 순서로 정렬
tfidf_df = tfidf_df.sort_values(by='counts', ascending=False).reset_index(drop=True)


tfidf_df

In [ ]:
## 한 번 해본 것

for i in tfidf_df['words']:
    if i in new:
        count = tfidf_df.loc[tfidf_df['words'] == i, 'counts'].values[0]
        print(f"The count for '{i}' is: {count}")

# 4. 워드 클라우드

## 필요한 모듈 임포트

In [ ]:
from wordcloud import WordCloud # 워드클라우드 제작 라이브러리
import pandas as pd # 데이터 프레임 라이브러리
import numpy as np # 행렬 라이브러리
import matplotlib.pyplot as plt # 워드클라우드 시각화 라이브러리
%matplotlib inline

import konlpy

## WordCloud 클래스의 객체 생성

참고 사이트: https://serendipity77.tistory.com/entry/영화동감-영화동감-댓글-리뷰-워드-클라우드wordcloud로-만들어보기파이썬-Python 

[엑셀 자동화로 칼퇴하는 김대리의 고군분투기:티스토리]

In [ ]:
font_path='C:\\Windows\\Fonts\\malgun.ttf'

### [body WordCloud]

In [ ]:
## 데이터 프레임을 딕셔너리 형태로 변환해야 함

dic_word = all_words_body.set_index('word').to_dict()['count']
dic_word

In [ ]:
## colormap 참고 사이트: https://wonhwa.tistory.com/20

wc = WordCloud(random_state = 123, font_path = font_path, width = 400,
               height = 400, background_color = 'black',
               colormap = 'Purples')
               ## 'Blues': 파란색 그라데이션
               ## 'BuGn': 청록색
               ## 'BuPu': 진보라 ~ 연한 파란색 ~ 하얀색 
               ## 'GnBu': 진파랑 ~ 연한 초록색 ~ 하얀색
               ## 'Greys': 검은색 ~ 회색 ~ 하얀색
               ## 'OrRd': 진빨강 ~ 연한 오렌지 ~ 하얀색
               ## 'Pastel1': 파스텔 색
               ## 'Pastel2': 파스텔 색
               ## 'PuBu': 진파랑 ~ 핑크 ~ 하얀색
               ## 'PuRd': 자주 ~ 핑크 ~ 하얀색
               ## 'Purples': 보라색
               ## 'RdPu': 보라 ~ 자주 ~ 핑크
               ## 'Reds': 빨강
               ## 'Wistia': 주황 ~ 형광 노랑
               ## 'YlGnBr': 파랑 ~ 초록 ~ 노랑


img_wordcloud = wc.generate_from_frequencies(dic_word)

plt.figure(figsize = (10, 10)) # 크기 지정하기
plt.axis('off') # 축 없애기
plt.imshow(img_wordcloud) # 결과 보여주기